# DeepFactor - Experiments

### Deep Factor (Wang et al.,  2019)

Hybrid method that uses *local* methods that are specific to each time series in the dataset as well as a *global* representation trained on the entire data. The latter is a RNN to learn universal pattern. The current version uses another RNN as local model -> **DeepFactor-RNN (DF-RNN)**

- [`DeepFactorEstimator`](http://gluon-ts.mxnet.io/master/api/gluonts/gluonts.model.deep_factor.html?highlight=deepfactorestimator#gluonts.model.deep_factor.DeepFactorEstimator)
- [Wang et al. (2019)](https://arxiv.org/abs/1905.12417)

In [1]:
# if version does not include DeepState, uncomment the following:
# !pip install git+https://github.com/awslabs/gluon-ts.git

In [2]:
import mxnet
mxnet.__version__

'1.4.1'

In [3]:
import gluonts
gluonts.__version__

'0.4.1'

In [4]:
import pprint
from functools import partial

# standard imports
import numpy as np
import pandas as pd

# vis imports 
%matplotlib inline
import matplotlib.pyplot as plt

# json
import json

# gluon data 
from gluonts.dataset.repository.datasets import get_dataset, dataset_recipes
from gluonts.dataset.util import to_pandas

# gluon imports
from gluonts.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator

# gluon distribution
from gluonts.distribution.piecewise_linear import PiecewiseLinearOutput
from gluonts.distribution.gaussian import GaussianOutput

# model imports 
# from gluonts.model.deepstate import DeepStateEstimator
from gluonts.model.deep_factor import DeepFactorEstimator
# from gluonts.model.deepar import DeepAREstimator

import mxnet as mx
from pprint import pprint

INFO:root:Using CPU


In [5]:
def main(seed=42, data="m4_daily", epochs=100, batches=50):
    
    mx.random.seed(seed)
    np.random.seed(seed)
    
    dataset = get_dataset(data)
    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=epochs,
        num_batches_per_epoch=batches,
        learning_rate=1e-3,
    )

    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepFactorEstimator(
        trainer=trainer,
        cardinality=[cardinality],
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
#         num_layers_global=2,
#         num_layers_local=2
    )

    predictor = estimator.train(dataset.train)

    forecast_it, ts_it = make_evaluation_predictions(
        dataset.test, predictor=predictor, num_samples=100
    )

    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series=len(dataset.test)
    )
    metrics = ["MASE", "sMAPE", "MSIS", "wQuantileLoss[0.5]", "wQuantileLoss[0.9]"]
    output = {key: round(value, 8) for key, value in agg_metrics.items() if key in metrics}
    df = pd.DataFrame([output])

    return(df)

# if __name__ == "__main__":
#     simple_main()

In [19]:
%%time
if __name__ == "__main__":
    df = main(seed=42, data="m4_quarterly", epochs=300, batches=100)
df

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_quarterly.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 100/100 [00:04<00:00, 23.53it/s, avg_epoch_loss=3.8e+7]
INFO:root:Epoch[0] Elapsed time 4.338 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=37973715.120000
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 100/100 [00:03<00:00, 26.49it/s, avg_epoch_loss=1.76e+7]
INFO:root:Epoch[1] Elapsed time 3.789 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=17629980.637500
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 100/100 [00:04<00:00, 22.72it/s, avg_epoch_loss=4.49e+6]
INFO:root:Epoch[2] Elapsed time 4.422 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=4493061.555000
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 100/100 [00:03<00:00, 26.44it/s, avg_epoch_loss=2.48e+6]
INFO:root:Epoch[3] Elapsed time 3.792 seconds
INFO:root:Epoch[3] Evalu

INFO:root:Epoch[34] Evaluation metric 'epoch_loss'=20002.011411
INFO:root:Epoch[35] Learning rate is 0.001
100%|██████████| 100/100 [00:03<00:00, 27.16it/s, avg_epoch_loss=1.96e+4]
INFO:root:Epoch[35] Elapsed time 3.696 seconds
INFO:root:Epoch[35] Evaluation metric 'epoch_loss'=19607.581646
INFO:root:Epoch[36] Learning rate is 0.001
100%|██████████| 100/100 [00:03<00:00, 30.26it/s, avg_epoch_loss=2.2e+4]
INFO:root:Epoch[36] Elapsed time 3.312 seconds
INFO:root:Epoch[36] Evaluation metric 'epoch_loss'=21995.715850
INFO:root:Epoch[37] Learning rate is 0.001
100%|██████████| 100/100 [00:03<00:00, 29.67it/s, avg_epoch_loss=2.15e+4]
INFO:root:Epoch[37] Elapsed time 3.379 seconds
INFO:root:Epoch[37] Evaluation metric 'epoch_loss'=21494.967568
INFO:root:Epoch[38] Learning rate is 0.001
100%|██████████| 100/100 [00:03<00:00, 26.19it/s, avg_epoch_loss=2.66e+4]
INFO:root:Epoch[38] Elapsed time 3.825 seconds
INFO:root:Epoch[38] Evaluation metric 'epoch_loss'=26609.815029
INFO:root:Epoch[39] Learn

INFO:root:Epoch[70] Learning rate is 0.00025
100%|██████████| 100/100 [00:03<00:00, 27.05it/s, avg_epoch_loss=2.21e+4]
INFO:root:Epoch[70] Elapsed time 3.724 seconds
INFO:root:Epoch[70] Evaluation metric 'epoch_loss'=22094.744316
INFO:root:Epoch[71] Learning rate is 0.00025
100%|██████████| 100/100 [00:03<00:00, 25.51it/s, avg_epoch_loss=1.8e+4]
INFO:root:Epoch[71] Elapsed time 3.930 seconds
INFO:root:Epoch[71] Evaluation metric 'epoch_loss'=17997.306372
INFO:root:Epoch[72] Learning rate is 0.00025
100%|██████████| 100/100 [00:03<00:00, 28.24it/s, avg_epoch_loss=2.11e+4]
INFO:root:Epoch[72] Elapsed time 3.548 seconds
INFO:root:Epoch[72] Evaluation metric 'epoch_loss'=21103.143867
INFO:root:Epoch[73] Learning rate is 0.00025
100%|██████████| 100/100 [00:03<00:00, 30.62it/s, avg_epoch_loss=2.65e+4]
INFO:root:Epoch[73] Elapsed time 3.273 seconds
INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=26483.779990
INFO:root:Epoch[74] Learning rate is 0.00025
100%|██████████| 100/100 [00:03<00:0

100%|██████████| 100/100 [00:04<00:00, 20.82it/s, avg_epoch_loss=1.14e+4]
INFO:root:Epoch[105] Elapsed time 4.812 seconds
INFO:root:Epoch[105] Evaluation metric 'epoch_loss'=11361.183433
INFO:root:Epoch[106] Learning rate is 0.000125
100%|██████████| 100/100 [00:04<00:00, 20.63it/s, avg_epoch_loss=1.38e+4]
INFO:root:Epoch[106] Elapsed time 4.858 seconds
INFO:root:Epoch[106] Evaluation metric 'epoch_loss'=13820.263148
INFO:root:Epoch[107] Learning rate is 0.000125
100%|██████████| 100/100 [00:04<00:00, 20.67it/s, avg_epoch_loss=7.27e+3]
INFO:root:Epoch[107] Elapsed time 4.849 seconds
INFO:root:Epoch[107] Evaluation metric 'epoch_loss'=7270.879820
INFO:root:Epoch[108] Learning rate is 0.000125
100%|██████████| 100/100 [00:04<00:00, 20.71it/s, avg_epoch_loss=6.16e+3]
INFO:root:Epoch[108] Elapsed time 4.837 seconds
INFO:root:Epoch[108] Evaluation metric 'epoch_loss'=6157.893865
INFO:root:Epoch[109] Learning rate is 0.000125
100%|██████████| 100/100 [00:08<00:00, 11.14it/s, avg_epoch_loss=8

INFO:root:Epoch[140] Learning rate is 6.25e-05
100%|██████████| 100/100 [00:09<00:00, 10.72it/s, avg_epoch_loss=3.47e+3]
INFO:root:Epoch[140] Elapsed time 9.342 seconds
INFO:root:Epoch[140] Evaluation metric 'epoch_loss'=3470.120278
INFO:root:Epoch[141] Learning rate is 6.25e-05
100%|██████████| 100/100 [00:07<00:00, 13.93it/s, avg_epoch_loss=1.72e+3]
INFO:root:Epoch[141] Elapsed time 7.192 seconds
INFO:root:Epoch[141] Evaluation metric 'epoch_loss'=1716.685999
INFO:root:Epoch[142] Learning rate is 6.25e-05
100%|██████████| 100/100 [00:06<00:00, 16.25it/s, avg_epoch_loss=1.12e+3]
INFO:root:Epoch[142] Elapsed time 6.160 seconds
INFO:root:Epoch[142] Evaluation metric 'epoch_loss'=1116.372211
INFO:root:Epoch[143] Learning rate is 6.25e-05
100%|██████████| 100/100 [00:04<00:00, 20.69it/s, avg_epoch_loss=1.84e+3]
INFO:root:Epoch[143] Elapsed time 4.840 seconds
INFO:root:Epoch[143] Evaluation metric 'epoch_loss'=1843.322042
INFO:root:Epoch[144] Learning rate is 6.25e-05
100%|██████████| 100/

...

INFO:root:Epoch[211] Elapsed time 4.311 seconds
INFO:root:Epoch[211] Evaluation metric 'epoch_loss'=632.154127
INFO:root:Epoch[212] Learning rate is 5e-05
100%|██████████| 100/100 [00:11<00:00,  8.50it/s, avg_epoch_loss=618]
INFO:root:Epoch[212] Elapsed time 11.779 seconds
INFO:root:Epoch[212] Evaluation metric 'epoch_loss'=618.370398
INFO:root:Epoch[213] Learning rate is 5e-05
100%|██████████| 100/100 [00:04<00:00, 22.51it/s, avg_epoch_loss=573]
INFO:root:Epoch[213] Elapsed time 4.452 seconds
INFO:root:Epoch[213] Evaluation metric 'epoch_loss'=572.932964
INFO:root:Epoch[214] Learning rate is 5e-05
100%|██████████| 100/100 [00:03<00:00, 28.82it/s, avg_epoch_loss=391]
INFO:root:Epoch[214] Elapsed time 3.481 seconds
INFO:root:Epoch[214] Evaluation metric 'epoch_loss'=391.000989
INFO:root:Epoch[215] Learning rate is 5e-05
100%|██████████| 100/100 [00:03<00:00, 27.63it/s, avg_epoch_loss=737]
INFO:root:Epoch[215] Elapsed time 3.628 seconds
INFO:root:Epoch[215] Evaluation metric 'epoch_loss'

INFO:root:Epoch[247] Evaluation metric 'epoch_loss'=330.156920
INFO:root:Epoch[248] Learning rate is 5e-05
100%|██████████| 100/100 [00:05<00:00, 17.60it/s, avg_epoch_loss=674]
INFO:root:Epoch[248] Elapsed time 5.690 seconds
INFO:root:Epoch[248] Evaluation metric 'epoch_loss'=673.544100
INFO:root:Epoch[249] Learning rate is 5e-05
100%|██████████| 100/100 [00:05<00:00, 18.91it/s, avg_epoch_loss=358]
INFO:root:Epoch[249] Elapsed time 5.302 seconds
INFO:root:Epoch[249] Evaluation metric 'epoch_loss'=357.944922
INFO:root:Epoch[250] Learning rate is 5e-05
100%|██████████| 100/100 [00:04<00:00, 21.94it/s, avg_epoch_loss=625]
INFO:root:Epoch[250] Elapsed time 4.570 seconds
INFO:root:Epoch[250] Evaluation metric 'epoch_loss'=625.215608
INFO:root:Epoch[251] Learning rate is 5e-05
100%|██████████| 100/100 [00:03<00:00, 27.20it/s, avg_epoch_loss=399]
INFO:root:Epoch[251] Elapsed time 3.686 seconds
INFO:root:Epoch[251] Evaluation metric 'epoch_loss'=399.082029
INFO:root:Epoch[252] Learning rate is

INFO:root:Epoch[284] Learning rate is 5e-05
100%|██████████| 100/100 [00:04<00:00, 24.68it/s, avg_epoch_loss=516]
INFO:root:Epoch[284] Elapsed time 4.066 seconds
INFO:root:Epoch[284] Evaluation metric 'epoch_loss'=516.395899
INFO:root:Epoch[285] Learning rate is 5e-05
100%|██████████| 100/100 [00:04<00:00, 20.06it/s, avg_epoch_loss=465]
INFO:root:Epoch[285] Elapsed time 4.995 seconds
INFO:root:Epoch[285] Evaluation metric 'epoch_loss'=464.723732
INFO:root:Epoch[286] Learning rate is 5e-05
100%|██████████| 100/100 [00:05<00:00, 19.53it/s, avg_epoch_loss=648]
INFO:root:Epoch[286] Elapsed time 5.131 seconds
INFO:root:Epoch[286] Evaluation metric 'epoch_loss'=648.475747
INFO:root:Epoch[287] Learning rate is 5e-05
100%|██████████| 100/100 [00:04<00:00, 22.36it/s, avg_epoch_loss=608]
INFO:root:Epoch[287] Elapsed time 4.480 seconds
INFO:root:Epoch[287] Evaluation metric 'epoch_loss'=607.649434
INFO:root:Epoch[288] Learning rate is 5e-05
100%|██████████| 100/100 [00:03<00:00, 26.96it/s, avg_ep

Wall time: 43min 12s


,MASE,sMAPE,MSIS,wQuantileLoss[0.5],wQuantileLoss[0.9]
0,3.774044,0.239329,141.181044,0.231349,0.262572


In [20]:
%%time
if __name__ == "__main__":
    df = main(seed=42, data="m4_quarterly", epochs=400, batches=100)
df

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_quarterly.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 100/100 [00:07<00:00, 14.02it/s, avg_epoch_loss=3.8e+7]
INFO:root:Epoch[0] Elapsed time 7.148 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=37973715.120000
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 100/100 [00:16<00:00,  6.05it/s, avg_epoch_loss=1.76e+7]
INFO:root:Epoch[1] Elapsed time 16.547 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=17629980.637500
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 100/100 [00:06<00:00, 15.22it/s, avg_epoch_loss=4.49e+6]
INFO:root:Epoch[2] Elapsed time 6.580 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=4493061.555000
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 100/100 [00:05<00:00, 17.59it/s, avg_epoch_loss=2.48e+6]
INFO:root:Epoch[3] Elapsed time 5.694 seconds
INFO:root:Epoch[3] Eval

INFO:root:Epoch[34] Evaluation metric 'epoch_loss'=20002.011411
INFO:root:Epoch[35] Learning rate is 0.001
100%|██████████| 100/100 [00:03<00:00, 27.23it/s, avg_epoch_loss=1.96e+4]
INFO:root:Epoch[35] Elapsed time 3.679 seconds
INFO:root:Epoch[35] Evaluation metric 'epoch_loss'=19607.581646
INFO:root:Epoch[36] Learning rate is 0.001
100%|██████████| 100/100 [00:06<00:00, 16.20it/s, avg_epoch_loss=2.2e+4]
INFO:root:Epoch[36] Elapsed time 6.188 seconds
INFO:root:Epoch[36] Evaluation metric 'epoch_loss'=21995.715850
INFO:root:Epoch[37] Learning rate is 0.001
100%|██████████| 100/100 [00:04<00:00, 20.28it/s, avg_epoch_loss=2.15e+4]
INFO:root:Epoch[37] Elapsed time 4.940 seconds
INFO:root:Epoch[37] Evaluation metric 'epoch_loss'=21494.967568
INFO:root:Epoch[38] Learning rate is 0.001
100%|██████████| 100/100 [00:04<00:00, 23.79it/s, avg_epoch_loss=2.66e+4]
INFO:root:Epoch[38] Elapsed time 4.206 seconds
INFO:root:Epoch[38] Evaluation metric 'epoch_loss'=26609.815029
INFO:root:Epoch[39] Learn

INFO:root:Epoch[70] Learning rate is 0.00025
100%|██████████| 100/100 [00:04<00:00, 23.95it/s, avg_epoch_loss=2.21e+4]
INFO:root:Epoch[70] Elapsed time 4.185 seconds
INFO:root:Epoch[70] Evaluation metric 'epoch_loss'=22094.744316
INFO:root:Epoch[71] Learning rate is 0.00025
100%|██████████| 100/100 [00:05<00:00, 16.78it/s, avg_epoch_loss=1.8e+4]
INFO:root:Epoch[71] Elapsed time 5.972 seconds
INFO:root:Epoch[71] Evaluation metric 'epoch_loss'=17997.306372
INFO:root:Epoch[72] Learning rate is 0.00025
100%|██████████| 100/100 [00:16<00:00,  6.18it/s, avg_epoch_loss=2.11e+4]
INFO:root:Epoch[72] Elapsed time 16.198 seconds
INFO:root:Epoch[72] Evaluation metric 'epoch_loss'=21103.143867
INFO:root:Epoch[73] Learning rate is 0.00025
100%|██████████| 100/100 [00:08<00:00, 12.02it/s, avg_epoch_loss=2.65e+4]
INFO:root:Epoch[73] Elapsed time 8.331 seconds
INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=26483.779990
INFO:root:Epoch[74] Learning rate is 0.00025
100%|██████████| 100/100 [00:07<00:

100%|██████████| 100/100 [00:05<00:00, 19.48it/s, avg_epoch_loss=1.14e+4]
INFO:root:Epoch[105] Elapsed time 5.143 seconds
INFO:root:Epoch[105] Evaluation metric 'epoch_loss'=11361.183433
INFO:root:Epoch[106] Learning rate is 0.000125
100%|██████████| 100/100 [00:05<00:00, 17.93it/s, avg_epoch_loss=1.38e+4]
INFO:root:Epoch[106] Elapsed time 5.589 seconds
INFO:root:Epoch[106] Evaluation metric 'epoch_loss'=13820.263148
INFO:root:Epoch[107] Learning rate is 0.000125
100%|██████████| 100/100 [00:07<00:00, 13.51it/s, avg_epoch_loss=7.27e+3]
INFO:root:Epoch[107] Elapsed time 7.413 seconds
INFO:root:Epoch[107] Evaluation metric 'epoch_loss'=7270.879820
INFO:root:Epoch[108] Learning rate is 0.000125
100%|██████████| 100/100 [00:11<00:00,  8.57it/s, avg_epoch_loss=6.16e+3]
INFO:root:Epoch[108] Elapsed time 11.679 seconds
INFO:root:Epoch[108] Evaluation metric 'epoch_loss'=6157.893865
INFO:root:Epoch[109] Learning rate is 0.000125
100%|██████████| 100/100 [00:06<00:00, 16.16it/s, avg_epoch_loss=

INFO:root:Epoch[140] Learning rate is 6.25e-05
100%|██████████| 100/100 [00:04<00:00, 20.43it/s, avg_epoch_loss=3.47e+3]
INFO:root:Epoch[140] Elapsed time 4.909 seconds
INFO:root:Epoch[140] Evaluation metric 'epoch_loss'=3470.120278
INFO:root:Epoch[141] Learning rate is 6.25e-05
100%|██████████| 100/100 [00:04<00:00, 20.73it/s, avg_epoch_loss=1.72e+3]
INFO:root:Epoch[141] Elapsed time 4.849 seconds
INFO:root:Epoch[141] Evaluation metric 'epoch_loss'=1716.685999
INFO:root:Epoch[142] Learning rate is 6.25e-05
100%|██████████| 100/100 [00:06<00:00, 15.55it/s, avg_epoch_loss=1.12e+3]
INFO:root:Epoch[142] Elapsed time 6.449 seconds
INFO:root:Epoch[142] Evaluation metric 'epoch_loss'=1116.372211
INFO:root:Epoch[143] Learning rate is 6.25e-05
100%|██████████| 100/100 [00:06<00:00, 14.32it/s, avg_epoch_loss=1.84e+3]
INFO:root:Epoch[143] Elapsed time 6.995 seconds
INFO:root:Epoch[143] Evaluation metric 'epoch_loss'=1843.322042
INFO:root:Epoch[144] Learning rate is 6.25e-05
100%|██████████| 100/

...

100%|██████████| 100/100 [00:03<00:00, 29.70it/s, avg_epoch_loss=553]
INFO:root:Epoch[320] Elapsed time 3.372 seconds
INFO:root:Epoch[320] Evaluation metric 'epoch_loss'=553.425633
INFO:root:Epoch[321] Learning rate is 5e-05
100%|██████████| 100/100 [00:03<00:00, 31.97it/s, avg_epoch_loss=550]
INFO:root:Epoch[321] Elapsed time 3.135 seconds
INFO:root:Epoch[321] Evaluation metric 'epoch_loss'=550.281490
INFO:root:Epoch[322] Learning rate is 5e-05
100%|██████████| 100/100 [00:03<00:00, 32.43it/s, avg_epoch_loss=382]
INFO:root:Epoch[322] Elapsed time 3.088 seconds
INFO:root:Epoch[322] Evaluation metric 'epoch_loss'=381.796892
INFO:root:Epoch[323] Learning rate is 5e-05
100%|██████████| 100/100 [00:03<00:00, 32.23it/s, avg_epoch_loss=597]
INFO:root:Epoch[323] Elapsed time 3.122 seconds
INFO:root:Epoch[323] Evaluation metric 'epoch_loss'=597.494691
INFO:root:Epoch[324] Learning rate is 5e-05
100%|██████████| 100/100 [00:03<00:00, 32.57it/s, avg_epoch_loss=335]
INFO:root:Epoch[324] Elapsed t

INFO:root:Epoch[356] Elapsed time 4.272 seconds
INFO:root:Epoch[356] Evaluation metric 'epoch_loss'=389.045661
INFO:root:Epoch[357] Learning rate is 5e-05
100%|██████████| 100/100 [00:03<00:00, 30.47it/s, avg_epoch_loss=383]
INFO:root:Epoch[357] Elapsed time 3.302 seconds
INFO:root:Epoch[357] Evaluation metric 'epoch_loss'=382.837191
INFO:root:Epoch[358] Learning rate is 5e-05
100%|██████████| 100/100 [00:03<00:00, 32.62it/s, avg_epoch_loss=710]
INFO:root:Epoch[358] Elapsed time 3.065 seconds
INFO:root:Epoch[358] Evaluation metric 'epoch_loss'=709.953217
INFO:root:Epoch[359] Learning rate is 5e-05
100%|██████████| 100/100 [00:03<00:00, 32.44it/s, avg_epoch_loss=378]
INFO:root:Epoch[359] Elapsed time 3.086 seconds
INFO:root:Epoch[359] Evaluation metric 'epoch_loss'=377.604018
INFO:root:Epoch[360] Learning rate is 5e-05
100%|██████████| 100/100 [00:03<00:00, 32.61it/s, avg_epoch_loss=423]
INFO:root:Epoch[360] Elapsed time 3.070 seconds
INFO:root:Epoch[360] Evaluation metric 'epoch_loss'=

INFO:root:Epoch[392] Evaluation metric 'epoch_loss'=391.882969
INFO:root:Epoch[393] Learning rate is 5e-05
100%|██████████| 100/100 [00:04<00:00, 23.87it/s, avg_epoch_loss=451]
INFO:root:Epoch[393] Elapsed time 4.210 seconds
INFO:root:Epoch[393] Evaluation metric 'epoch_loss'=451.042739
INFO:root:Epoch[394] Learning rate is 5e-05
100%|██████████| 100/100 [00:03<00:00, 26.80it/s, avg_epoch_loss=352]
INFO:root:Epoch[394] Elapsed time 3.755 seconds
INFO:root:Epoch[394] Evaluation metric 'epoch_loss'=352.434059
INFO:root:Epoch[395] Learning rate is 5e-05
100%|██████████| 100/100 [00:03<00:00, 32.60it/s, avg_epoch_loss=489]
INFO:root:Epoch[395] Elapsed time 3.068 seconds
INFO:root:Epoch[395] Evaluation metric 'epoch_loss'=489.486031
INFO:root:Epoch[396] Learning rate is 5e-05
100%|██████████| 100/100 [00:03<00:00, 33.20it/s, avg_epoch_loss=487]
INFO:root:Epoch[396] Elapsed time 3.016 seconds
INFO:root:Epoch[396] Evaluation metric 'epoch_loss'=486.927367
INFO:root:Epoch[397] Learning rate is

Wall time: 40min 23s


,MASE,sMAPE,MSIS,wQuantileLoss[0.5],wQuantileLoss[0.9]
0,4.647202,0.258783,174.731894,0.279884,0.169492


In [21]:
%%time
if __name__ == "__main__":
    df = main(seed=42, data="m4_quarterly", epochs=500, batches=100)
df

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_quarterly.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 100/100 [00:03<00:00, 25.76it/s, avg_epoch_loss=3.8e+7]
INFO:root:Epoch[0] Elapsed time 3.891 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=37973715.120000
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 100/100 [00:03<00:00, 29.83it/s, avg_epoch_loss=1.76e+7]
INFO:root:Epoch[1] Elapsed time 3.372 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=17629980.637500
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 100/100 [00:03<00:00, 31.15it/s, avg_epoch_loss=4.49e+6]
INFO:root:Epoch[2] Elapsed time 3.215 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=4493061.555000
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 100/100 [00:03<00:00, 29.79it/s, avg_epoch_loss=2.48e+6]
INFO:root:Epoch[3] Elapsed time 3.361 seconds
INFO:root:Epoch[3] Evalu

INFO:root:Epoch[34] Evaluation metric 'epoch_loss'=20002.011411
INFO:root:Epoch[35] Learning rate is 0.001
100%|██████████| 100/100 [00:03<00:00, 32.16it/s, avg_epoch_loss=1.96e+4]
INFO:root:Epoch[35] Elapsed time 3.133 seconds
INFO:root:Epoch[35] Evaluation metric 'epoch_loss'=19607.581646
INFO:root:Epoch[36] Learning rate is 0.001
100%|██████████| 100/100 [00:03<00:00, 29.97it/s, avg_epoch_loss=2.2e+4]
INFO:root:Epoch[36] Elapsed time 3.344 seconds
INFO:root:Epoch[36] Evaluation metric 'epoch_loss'=21995.715850
INFO:root:Epoch[37] Learning rate is 0.001
100%|██████████| 100/100 [00:03<00:00, 31.08it/s, avg_epoch_loss=2.15e+4]
INFO:root:Epoch[37] Elapsed time 3.237 seconds
INFO:root:Epoch[37] Evaluation metric 'epoch_loss'=21494.967568
INFO:root:Epoch[38] Learning rate is 0.001
100%|██████████| 100/100 [00:03<00:00, 30.16it/s, avg_epoch_loss=2.66e+4]
INFO:root:Epoch[38] Elapsed time 3.320 seconds
INFO:root:Epoch[38] Evaluation metric 'epoch_loss'=26609.815029
INFO:root:Epoch[39] Learn

KeyboardInterrupt: 

In [22]:
!jupyter nbconvert --output-dir="./html_outputs" --to html DeepFactor01-DeepFactor_experiments_with_gluonts.ipynb

[NbConvertApp] Converting notebook DeepFactor01-DeepFactor_experiments_with_gluonts.ipynb to html
[NbConvertApp] Writing 466427 bytes to ./html_outputs\DeepFactor01-DeepFactor_experiments_with_gluonts.html


In [18]:
print("Done!")

Done!
